# Distribuições de Probabilidade e Amostragem

## Probabilidade de alguém ser usuário de uma operadora de telefonia

Qual é a probabilidade de uma pessoa no Brasil ser cliente da VIVO, CLARO ou TIM?

Fonte: https://sistemas.anatel.gov.br/anexar-api/publico/anexos/download/44aaef993685036f18cab2ea0b4561e2

Sendo que o percentual do mercado já foi calculado, iremos apenas normalizar o percentual para somar 1.

Se considerarmos a população do país de 213.421.037 habitantes (https://www.ibge.gov.br/), podemos multiplicar a probabilidade pela quantidade total de habitantes e inferir a quantidade de usuários de determinada operadora.

In [8]:
import pandas as pd

# Caminho do arquivo CSV
csv_path = 'Aula2/julio/Operadoradetelefonia-Pctmercado-Perodo.csv'

# Lê o CSV em um DataFrame
df = pd.read_csv(csv_path)

# Adiciona a coluna "P" como probabilidade (percentual dividido por 100)
df['P'] = df['Pct mercado'] / 100

# Exibe o DataFrame resultante
# print(df)

# Filtra apenas pelo período "2T2025"
filtered_df = df[df['Período'] == '2T2025']
print(filtered_df)

# Some todas as probabilidades de filtered_df
total_probability = filtered_df['P'].sum()
print(f'Probabilidade total deve somar 1: {total_probability}')

# Salve o DataFrame filtrado em um novo CSV
filtered_csv_path = 'Aula2/julio/Operadoradetelefonia-Pctmercado-2T2025.csv'
filtered_df.to_csv(filtered_csv_path, index=False)

   Operadora de telefonia  Pct mercado Período       P
1                    VIVO        38.50  2T2025  0.3850
3                   CLARO        32.52  2T2025  0.3252
5                     TIM        23.40  2T2025  0.2340
7                   ALGAR         1.68  2T2025  0.0168
9                  Datora         1.13  2T2025  0.0113
11                   SURF         0.93  2T2025  0.0093
13             Next Level         0.42  2T2025  0.0042
15              Teleports         0.32  2T2025  0.0032
17               BRISANET         0.09  2T2025  0.0009
19                   VERO         0.08  2T2025  0.0008
21               UNIFIQUE         0.06  2T2025  0.0006
23                  LIGGA         0.01  2T2025  0.0001
Probabilidade total deve somar 1: 0.9914000000000002


In [12]:
# Lê o CSV filtrado em um novo DataFrame
df = pd.read_csv(filtered_csv_path)

# crie uma nova coluna "Estimated Users" que multiplica a probabilidade "P" pela população total do Brasil (213421037)
population_brazil = 213421037
df['Estimated Users'] = df['P'] * population_brazil
print(df)
print('+82 bilhões de usuários da VIVO no Brasil')

   Operadora de telefonia  Pct mercado Período       P  Estimated Users
0                    VIVO        38.50  2T2025  0.3850     8.216710e+07
1                   CLARO        32.52  2T2025  0.3252     6.940452e+07
2                     TIM        23.40  2T2025  0.2340     4.994052e+07
3                   ALGAR         1.68  2T2025  0.0168     3.585473e+06
4                  Datora         1.13  2T2025  0.0113     2.411658e+06
5                    SURF         0.93  2T2025  0.0093     1.984816e+06
6              Next Level         0.42  2T2025  0.0042     8.963684e+05
7               Teleports         0.32  2T2025  0.0032     6.829473e+05
8                BRISANET         0.09  2T2025  0.0009     1.920789e+05
9                    VERO         0.08  2T2025  0.0008     1.707368e+05
10               UNIFIQUE         0.06  2T2025  0.0006     1.280526e+05
11                  LIGGA         0.01  2T2025  0.0001     2.134210e+04
+82 bilhões de usuários da VIVO no Brasil



No SGBD clickhouse, criar um DB IBGE

https://clickhouse.com/docs/knowledgebase/importing-geojason-with-nested-object-array#question

- carregar shape (geojson)
- fazer operações geoespacias
- carregar microdados censo
- fazer JOIN e mostrar distribuições

Descomprimir e mover para a pasta de leitura do clickhouse

```bash
sudo mv BR_Municipios_2024.geojson /var/lib/clickhouse/user_files
```

Depois de entrar no clickhouse-client:

```sql
DESCRIBE TABLE file('BR_Municipios_2024.geojson', 'JSON')
```

Vemos:
```bash
DESCRIBE TABLE file('BR_Municipios_2024.geojson', 'JSON')

Query id: 69b90589-8b43-4dbf-b461-b98de6c61636

Row 1:
──────
name:               type
type:               Nullable(String)
default_type:       
default_expression: 
comment:            
codec_expression:   
ttl_expression:     

Row 2:
──────
name:               name
type:               Nullable(String)
default_type:       
default_expression: 
comment:            
codec_expression:   
ttl_expression:     

Row 3:
──────
name:               features
type:               Array(Tuple(
    geometry Tuple(
        coordinates Array(Array(Array(Array(Nullable(Float64))))),
        type Nullable(String)),
    properties Tuple(
        AREA_KM2 Nullable(Float64),
        CD_CONCU Nullable(String),
        CD_MUN Nullable(String),
        CD_REGIA Nullable(String),
        CD_RGI Nullable(String),
        CD_RGINT Nullable(String),
        CD_UF Nullable(String),
        NM_CONCU Nullable(String),
        NM_MUN Nullable(String),
        NM_REGIA Nullable(String),
        NM_RGI Nullable(String),
        NM_RGINT Nullable(String),
        NM_UF Nullable(String),
        SIGLA_RG Nullable(String),
        SIGLA_UF Nullable(String)),
    type Nullable(String)))
default_type:       
default_expression: 
comment:            
codec_expression:   
ttl_expression:     

3 rows in set. Elapsed: 9.143 sec. 
```

Criar a tabela:

```sql
create database ibge;
use ibge;
CREATE TABLE br_municipios
(
    type String,
    name String,
    featureType String,
    
    -- Dados do município
    cd_mun String,
    nm_mun String,
    area_km2 Float64,
    
    -- Dados da UF
    cd_uf String,
    nm_uf String,
    sigla_uf String,
    
    -- Dados regionais
    cd_concu Nullable(String),
    nm_concu Nullable(String),
    cd_regia Nullable(String),
    nm_regia Nullable(String),
    cd_rgi Nullable(String),
    nm_rgi Nullable(String),
    cd_rgint Nullable(String),
    nm_rgint Nullable(String),
    sigla_rg Nullable(String),
    
    -- Geometria
    geometryType String,
    geometry MultiPolygon
)
ENGINE = MergeTree
ORDER BY (cd_uf, cd_mun);
```

Verificando antes de inserir.

```sql
SELECT
    type,
    name,
    features.type AS featureType,
    features.properties.CD_MUN AS cd_mun,
    features.properties.NM_MUN AS nm_mun,
    features.properties.SIGLA_UF AS sigla_uf,
    features.geometry.type AS geometryType
FROM file('BR_Municipios_2024.geojson', 'JSON')
ARRAY JOIN features
LIMIT 5;
```

Inserindo os dados:
```sql
INSERT INTO br_municipios
SELECT
    type,
    name,
    features.type AS featureType,
    
    -- Dados do município
    features.properties.CD_MUN AS cd_mun,
    features.properties.NM_MUN AS nm_mun,
    features.properties.AREA_KM2 AS area_km2,
    
    -- Dados da UF
    features.properties.CD_UF AS cd_uf,
    features.properties.NM_UF AS nm_uf,
    features.properties.SIGLA_UF AS sigla_uf,
    
    -- Dados regionais
    features.properties.CD_CONCU AS cd_concu,
    features.properties.NM_CONCU AS nm_concu,
    features.properties.CD_REGIA AS cd_regia,
    features.properties.NM_REGIA AS nm_regia,
    features.properties.CD_RGI AS cd_rgi,
    features.properties.NM_RGI AS nm_rgi,
    features.properties.CD_RGINT AS cd_rgint,
    features.properties.NM_RGINT AS nm_rgint,
    features.properties.SIGLA_RG AS sigla_rg,
    
    -- Geometria
    features.geometry.type AS geometryType,
    arrayMap(
        poly -> arrayMap(
            ring -> arrayMap(
                coord -> (coord[1], coord[2]),
                ring
            ),
            poly
        ),
        features.geometry.coordinates
    ) AS geometry
FROM file('BR_Municipios_2024.geojson', 'JSON')
ARRAY JOIN features;
```

Verificar:

```sql
-- Contar registros
SELECT count() FROM br_municipios;

-- Verificar o tipo da geometria
SELECT DISTINCT toTypeName(geometry) FROM br_municipios;

-- Ver alguns municípios por estado
SELECT sigla_uf, count() AS total_municipios
FROM br_municipios
GROUP BY sigla_uf
ORDER BY sigla_uf;
```